In [2]:
from scipy.fftpack import dct
import numpy as np
import scipy.io.wavfile
import matplotlib.pyplot as plt
import glob
import numpy as np
import scipy.io.wavfile
from scipy.fftpack import dct
from scipy.signal import savgol_filter
from PyEMD import EMD
import pywt
from scipy.signal import hilbert
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot

In [ ]:
root_path = str(input("enter the path of root directory"))
folder_path_list = list(glob.glob(str(root_path+"/*/")))

In [ ]:
folder_list

In [1]:
def pre_processing(sample_rate,signal):
    ##### Pre-Emphasis
    #set filter coefficient for pre-emphasis  filter coefficient (α) 
    pre_emphasis = 0.97 #typical value need to be set                                                 **
    
    emphasized_signal = numpy.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])
    
    ##### sav_gol filter                                                                               **
    #link:- https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.signal.savgol_filter.html
    np.set_printoptions(precision=2)  # For compact display.
    after_savgol_signal = savgol_filter(emphasized_signal, 5, 2, mode = 'nearest')
    # 5 is the window length and 2 is the polynomial degree 
    
    
    ##### emd
    #link :- https://github.com/laszukdawid/PyEMD
    emd = EMD()        #basic emd
    imfs = emd(after_savgol_signal)
    
    
    frames_list = list()
    for i in imfs:
        """
        #### Framing
        #Typical frame sizes in speech processing range from 20 ms to 40 ms 
        #with 50% (+/-10%) overlap between consecutive frames. 
        #Popular settings are 25 ms for the frame size,  and a 10 ms stride (15 ms overlap), 
        #link :- https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html
        frame_size = 0.025
        frame_stride = 0.01
        frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
        signal_length = len(i)
        frame_length = int(round(frame_length))
        frame_step = int(round(frame_step))
        num_frames = int(numpy.ceil(float(numpy.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame

        pad_signal_length = num_frames * frame_step + frame_length
        z = numpy.zeros((pad_signal_length - signal_length))
        pad_signal = numpy.append(i, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

        indices = numpy.tile(numpy.arange(0, frame_length), (num_frames, 1)) + numpy.tile(numpy.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
        frames = pad_signal[indices.astype(numpy.int32, copy=False)]
        """
        max_pcc = 0
        coef = np.corrcoef(signal, i)[0, 1]
        if(coef > max_pcc):
            max_pcc = coef
            max_imf = i
    #thresholding https://pywavelets.readthedocs.io/en/latest/ref/thresholding-functions.html
    pywt.threshold(max_imf, 2, 'soft')
    
            
    return frames_list,imfs
    
        ######
        
        
    
    

In [2]:
def gp1(imfs):
    insta_amp_list = list()
    insta_freq_list = list()
    for i in imfs:
        ### hilbert transform
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.hilbert.html
        # https://dsp.stackexchange.com/questions/46291/why-is-scipy-implementation-of-hilbert-function-different-from-matlab-implemen
        #post_hibert = hilbert(i)
        analytic_signal = hilbert(i)
        amplitude_envelope = np.abs(analytic_signal)
        instantaneous_phase = np.unwrap(np.angle(analytic_signal))
        instantaneous_frequency = (np.diff(instantaneous_phase) /(2.0*np.pi) * fs)
        insta_amp_list.append(amplitude_envelope[:13]) 
        insta_freq_list.append(instantaneous_frequency[:13])
        
        #task 2 hilbert spectrum
        sensor = np.loadtxt('signal.txt')
        plt.plot(sensor)

        index = 1
        plt.plot(analytical_signal.real)
        plt.plot(analytical_signal.imag)
        plt.plot(amplitude_envelope)
        file_name = str(index)+"imf_spectrum"
        plt.savefig(file_name)
        index = index +1
        return insta_amp_list,insta_freq_list

In [ ]:


def gp2(imfs):
    
    ##### DCT
    # https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.fftpack.dct.html
    after_dct = dct(i)
    first_13 = after_dct[0:13]
    first_diff = np.diff(first_13,1)
    second_diff = np.diff(first_13,2)
    return first_diff,second_diff

In [ ]:
for i in folder_list:
    sampleaudio_path_list = list(glob.glob(str(folder_list+"/*.wav")))
    for i in range(0,len(sampleaudio_path_list)):
        sample_rate, signal = scipy.io.wavfile.read(sampleaudio_path_list[i])  # File assumed to be in the same directory
        frames,imfs = pre_processing(samplerate,signal) 
        final_list = list()
        for i in imfs:
            insta_amp_i, insta_freq_i = gp1(i)
            first_diff_i,second_diff_i = gp2(i)
            combined_vector = insta_amp_i+insta_freq_i+first_diff_i+second_diff_i 
            final_list.append(combined_vector)
        
        
    #code followes here....
    pass